# 1. Import modules

In [6]:
import sys
import os
import numpy as np
import gc

In [7]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from datetime import datetime
from tqdm import tqdm

# 2. Define the device for training

In [8]:
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"

# 3. Define model

In [9]:
def getVGGModel():
  vgg16 = models.vgg16_bn(weights=models.vgg.VGG16_BN_Weights.IMAGENET1K_V1)

  # Fix the conv layers parameters
  for conv_param in vgg16.features.parameters():
    conv_param.require_grad = False

  # Replace w/ new classification layers
  classifications = nn.Sequential(
    nn.Linear(25088,2048),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(2048,3)
  )

  vgg16.classifier = classifications

  return vgg16

In [10]:
model = getVGGModel()
    
model.to(DEVICE)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

# 4. Define hyperparameters

In [11]:
hp = {"lr":1e-5, "beta1":0.9, "beta2":0.999, "batch_size":16, "epochs":5}

# 5. Load dataset and define data augmentation strategy

In [12]:
def load_datasets(train_path, val_path, test_path):
  val_img_transform = transforms.Compose([transforms.Resize((244,244)),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  train_img_transform = transforms.Compose([transforms.RandomHorizontalFlip(), 
                                            transforms.RandomAffine(degrees=0, scale=(0.8,1.2), shear=0.2, translate=(0.2, 0.2)), 
                                            transforms.ToTensor(),
                                            transforms.Resize((244,244)), 
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

  train_dataset = datasets.ImageFolder(train_path, transform=train_img_transform)
  val_dataset = datasets.ImageFolder(val_path, transform=val_img_transform) 
  test_dataset = datasets.ImageFolder(test_path, transform=val_img_transform) if test_path is not None else None
  print(f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")
  return train_dataset, val_dataset, test_dataset
    
def construct_dataloaders(train_set, val_set, test_set, batch_size, shuffle=True):
  train_dataloader = torch.utils.data.DataLoader(train_set, batch_size, shuffle)
  val_dataloader = torch.utils.data.DataLoader(val_set, batch_size) 
  test_dataloader = torch.utils.data.DataLoader(test_aset, batch_size) if test_path is not None else None
  return train_dataloader, val_dataloader, test_dataloader

In [13]:
# Please specify the path to train, cross_validation, and test images below:
train_path, val_path, test_path = "/tmp/Dataset_2/Train/", "/tmp/Dataset_2/Validation/", None
train_set, val_set, test_set = load_datasets(train_path, val_path, test_path)
train_dataloader, val_dataloader, test_dataloader = construct_dataloaders(train_set, val_set, test_set, hp["batch_size"], True)

Train set size: 1322, Validation set size: 363


# 6. Define optimizer

In [14]:
opt = torch.optim.Adam(model.parameters(),lr=hp["lr"], betas=(hp["beta1"], hp["beta2"]))

# 7. Define loss function

In [15]:
loss_fn = nn.CrossEntropyLoss()

# 8. Train model

## 8.1 Define train function

In [16]:
@torch.no_grad()
def eval_model(data_loader, model, loss_fn, DEVICE):
  model.train(False)
  model.eval()
  loss, accuracy = 0.0, 0.0
  n = len(data_loader)

  for i, data in enumerate(data_loader):
    x,y = data
    x,y = x.to(DEVICE), y.to(DEVICE)
    pred = model(x)
    loss += loss_fn(pred, y)/len(x)
    pred_label = torch.argmax(pred, axis = 1)
    accuracy += torch.sum(pred_label == y)/len(x)

  return loss/n, accuracy/n

## 8.2 Define evaluation function

In [17]:
def train(train_loader, val_loader, model, opt, loss_fn, epochs, DEVICE):
  
  for epoch in range(epochs):
    model.train(True)
    
    print(f"Epoch {epoch+1}/{epochs}:")
    start_time = datetime.now()
    
    total_loss = 0
    total_accuracy = 0
    total_count = 0
    
    with tqdm(
        total=len(train_loader),
        bar_format='{l_bar}{bar:10}{r_bar}',
        desc=f'Epoch {epoch:3d}/{epochs:3d}',
        disable=False
    ) as t:
        for x, y in train_loader:
          x, y = x.to(DEVICE), y.to(DEVICE)
          pred = model(x)
          loss = loss_fn(pred,y)

          opt.zero_grad()
          loss.backward()
          opt.step()

          pred_label = torch.argmax(pred, axis=1)
          total_loss += loss
          total_accuracy += torch.sum(pred_label == y)
          total_count += len(x)
          t.set_postfix_str(
                    'loss: {:.4f}, acc: {:.2f}%'.format(
                        total_loss/total_count,
                        100*total_accuracy/total_count,
                    ),
                )
          t.update(1)
      
      
    end_time = datetime.now()    
    print(f"Time: {(end_time-start_time).seconds}s")

    val_loss, val_acc = eval_model(val_loader, model, loss_fn, DEVICE)
    print(f"Val loss: {val_loss}, Val accuracy: {val_acc}\n")

## 8.3 Start training

In [18]:
train(train_dataloader, val_dataloader, model, opt, loss_fn, hp["epochs"], DEVICE)

Epoch 1/5:


Epoch   0/  5: 100%|██████████| 83/83 [07:44<00:00,  5.60s/it, loss: 0.0611, acc: 53.63%]


Time: 464s
Val loss: 0.05047643184661865, Val accuracy: 0.6916996240615845

Epoch 2/5:


Epoch   1/  5: 100%|██████████| 83/83 [08:05<00:00,  5.85s/it, loss: 0.0467, acc: 67.85%]


Time: 485s
Val loss: 0.04201819747686386, Val accuracy: 0.7028161883354187

Epoch 3/5:


...

Time: 460s
Val loss: 0.04148910567164421, Val accuracy: 0.6810771226882935

Epoch 4/5:


Epoch   3/  5: 100%|██████████| 83/83 [08:20<00:00,  6.03s/it, loss: 0.0359, acc: 74.28%]


Time: 500s
Val loss: 0.039522796869277954, Val accuracy: 0.7000988125801086

Epoch 5/5:


Epoch   4/  5: 100%|██████████| 83/83 [07:41<00:00,  5.57s/it, loss: 0.0301, acc: 78.44%]


Time: 462s
Val loss: 0.03911827132105827, Val accuracy: 0.7245553135871887

